# Decoding latent descriptors and Symbolic Regression (SR) features

We continue the evaluation of the refractive index dataset now decoding the l-MM and l-OFM features, computing SISSO features from them and also decomposing GNN features into interpretable MM and OFM descriptors.

In [3]:
import pandas as pd
import json
from pymatgen.core import Structure

# --- Step 1: Load the CSV file containing JSON strings ---
file_path = 'data/df_ref_index.csv'
df = pd.read_csv(file_path, index_col=0)
print(f"Loaded {len(df)} records from {file_path}")

# --- Step 2: Convert JSON strings back to Structure objects ---
# We use json.loads to parse the string, then Structure.from_dict
print("Reconstructing Structure objects from JSON...")
df['structure'] = df['structure'].apply(
    lambda json_string: Structure.from_dict(json.loads(json_string))
)
# --- Step 3: Verification ---
print("✅ Reconstruction complete!")
print("First few datapoints:")
df.head()


Loaded 4022 records from data/df_ref_index.csv
Reconstructing Structure objects from JSON...
✅ Reconstruction complete!
First few datapoints:


,structure,ref_index
mp-624234,"[[0.67808954 1.32800354 5.90141888] Te, [1.500...",2.440483
mp-560478,"[[-0.62755181 6.55361247 9.268476 ] Ba, [4....",1.790685
mp-556346,"[[4.43332093 4.12714801 8.8721209 ] Pr, [ 1.40...",2.056131
mp-13676,"[[-0.14481557 3.41229366 4.12618551] O, [3.2...",2.023772
mp-7610,"[[ 0.12549448 3.01287591 -0.20434955] Li, [1....",1.745509


### (b) Test the different feature sets for prediction
We use scikit-learn models and XGBoost to evaluate the prediction performance of various feature combinations. 

In [29]:
## We need to separate in train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

### We will now decode the OFM and MM features
##### We change now to the primary_env since these models are MEGNet based.

In [3]:
from mattervial.interpreter.decoder import decode_ofm, decode_mm
import pandas as pd
features_l_ofm = pd.read_csv('data/ref_index_features_l_ofm.csv',index_col=0)
features_ofm = decode_ofm(features_l_ofm)
display(features_ofm.head())

features_l_mm = pd.read_csv('data/ref_index_features_l_mm.csv',index_col=0)
features_mm = decode_mm(features_l_mm)
display(features_mm.head())

/gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/mattervial-0.1.4-py3.9.egg/mattervial/packages/roost/core.py:12: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm



--- Decoding l-OFM data to OFM ---
Decoding data...
126/126 [==============================] - 0s 1ms/step
Inverse scaling data...
Final decoded shape: (4022, 943)
OFM Decoding complete.


/gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.3 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,OFM: s^1 - s^1,OFM: s^1 - s^2,OFM: s^1 - p^1,OFM: s^1 - p^2,OFM: s^1 - p^3,OFM: s^1 - p^4,OFM: s^1 - p^5,OFM: s^1 - p^6,OFM: s^1 - d^1,OFM: s^1 - d^2,...,OFM: f^14 - f^4,OFM: f^14 - f^5,OFM: f^14 - f^6,OFM: f^14 - f^7,OFM: f^14 - f^9,OFM: f^14 - f^10,OFM: f^14 - f^11,OFM: f^14 - f^12,OFM: f^14 - f^13,OFM: f^14 - f^14
mp-624234,0.004853,0.000857,0.000085,0.000055,-0.000181,-0.000690,0.000537,-0.000113,-0.001467,0.000315,...,0.000068,0.000158,0.000075,0.000035,0.000217,-0.000117,-0.000128,0.000119,0.000452,0.011769
mp-560478,0.003746,-0.001189,0.000044,-0.000410,0.000052,-0.001160,-0.000477,-0.000162,-0.000626,0.000400,...,0.000070,0.000109,0.000019,0.000102,0.000223,0.000148,-0.000049,0.000092,0.000260,0.003329
mp-556346,0.013634,0.015471,0.000332,-0.000045,0.001349,0.004699,0.003816,-0.000031,-0.000149,0.000385,...,0.000113,0.000157,0.000297,0.000162,0.000353,0.000200,-0.000017,0.000382,0.000612,0.006675
mp-13676,0.004263,-0.004251,0.001490,-0.000951,-0.000180,-0.004101,0.000157,-0.000158,-0.001031,0.000190,...,0.000138,0.000141,-0.000071,0.000066,-0.000071,-0.000065,-0.000155,-0.000084,0.000093,0.009424
mp-7610,0.028043,0.260847,0.000625,0.002234,0.000565,0.254620,-0.000074,-0.000205,-0.000727,0.000409,...,0.000086,0.000132,0.000097,0.000092,0.000261,0.000127,-0.000070,0.000147,0.000366,0.003896



--- Decoding l-MM data to MatMiner ---
Decoding data...
126/126 [==============================] - 1s 4ms/step
Inverse scaling data...
Final decoded shape: (4022, 1264)
MatMiner Decoding complete.


/gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.3 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,AtomicOrbitals|HOMO_character,AtomicOrbitals|HOMO_element,AtomicOrbitals|HOMO_energy,AtomicOrbitals|LUMO_character,AtomicOrbitals|LUMO_element,AtomicOrbitals|LUMO_energy,AtomicOrbitals|gap_AO,AtomicPackingEfficiency|mean simul. packing efficiency,AtomicPackingEfficiency|mean abs simul. packing efficiency,AtomicPackingEfficiency|dist from 1 clusters |APE| < 0.010,...,CrystalNNFingerprint|std_dev wt CN_19,CoulombMatrix|coulomb matrix eig 85,SineCoulombMatrix|sine coulomb matrix eig 102,SineCoulombMatrix|sine coulomb matrix eig 84,BondFractions|Nb - O bond frac.,CoulombMatrix|coulomb matrix eig 108,CoulombMatrix|coulomb matrix eig 89,CrystalNNFingerprint|mean wt CN_21,BondFractions|Sc - Sc bond frac.,BondFractions|B - B bond frac.
mp-624234,2.002619,13.404082,-0.330379,1.988082,36.567394,-0.274597,0.054147,0.015422,0.028403,0.078292,...,0.000826,58.794487,37.791401,33.448124,0.003362,22.843225,50.238373,0.000031,0.000250,-0.001563
mp-560478,2.108094,11.226305,-0.321251,2.104832,15.059876,-0.285261,0.034124,0.020605,0.044055,0.059758,...,0.000767,27.561857,28.173471,20.907290,0.004689,22.819960,25.934271,0.000084,0.000830,-0.001294
mp-556346,2.825146,28.565887,-0.300052,2.869596,30.779621,-0.292360,0.005223,0.025885,0.038368,0.094171,...,0.000853,45.564629,34.737995,21.138964,0.002391,30.183470,40.198139,0.000007,0.000825,-0.000988
mp-13676,2.107716,13.218966,-0.335132,2.040744,16.646544,-0.330378,0.002358,0.018726,0.033835,0.063567,...,0.000490,32.860935,23.876526,17.482370,0.001511,8.769359,23.613997,0.000010,-0.000214,-0.001562
mp-7610,2.299532,16.632030,-0.292702,2.331344,17.925247,-0.270479,0.019468,0.008792,0.024607,0.086277,...,0.001051,54.602692,39.468323,25.988585,0.005448,37.214554,36.975338,0.000057,0.001180,-0.000974


2025-09-11 20:01:32.422746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import pandas as pd
import numpy as np
import os, joblib
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('data/df_ref_index.csv',index_col=0)
# Load the feature dataframes
features_l_mm = pd.read_csv('./data/ref_index_features_l_mm.csv', index_col=0)
features_l_ofm = pd.read_csv('./data/ref_index_features_l_ofm.csv', index_col=0)
features_mvl = pd.read_csv('./data/ref_index_features_mvl.csv', index_col=0)
adjmegnet_features = pd.read_csv('./data/ref_index_features_adjmegnet.csv', index_col=0)
roost_gap_features = pd.read_csv('./data/ref_index_features_roost_gap.csv', index_col=0)
roost_eform_features = pd.read_csv('./data/ref_index_features_roost_eform.csv', index_col=0)
orb_features = pd.read_csv('./data/ref_index_features_orbv3.csv', index_col=0)


# Concatenate all features (aligns by index)
X_all = pd.concat([features_l_mm, features_l_ofm, features_mvl, adjmegnet_features, roost_gap_features, roost_eform_features, orb_features, features_ofm, features_mm], axis=1)
# X_all = pd.concat([ features_mvl, adjmegnet_features, roost_gap_features, roost_eform_features, orb_features, features_ofm, features_mm], axis=1)

## We need to separate in train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Align to train/test indexes and fill missing values
X_train = X_all.reindex(train_df.index).fillna(0)
X_test = X_all.reindex(test_df.index).fillna(0)

y_train = train_df['ref_index']
y_test = test_df['ref_index']

# Train XGBoost regressor
model = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
model.fit(X_train.values, y_train.values)

# Predict and report metrics
y_pred_train = model.predict(X_train.values)
y_pred_test = model.predict(X_test.values)

print("Train MAE:", mean_absolute_error(y_train, y_pred_train))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train)))
print("Train R2:", r2_score(y_train, y_pred_train))
print("---")
print("Test MAE:", mean_absolute_error(y_test, y_pred_test))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
print("Test R2:", r2_score(y_test, y_pred_test))

# Save model
os.makedirs('./models', exist_ok=True)
model_path = './models/xgb_best_and_decoded_features.joblib'
joblib.dump(model, model_path)
print("Model saved to", model_path)

# Show top 20 feature importances
fi = pd.DataFrame({
   'Feature': X_train.columns,
   'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 20 features:")
print(fi.head(20).to_string(index=False))


Train MAE: 0.0014650721163266699
Train RMSE: 0.001933045334780528
Train R2: 0.9999793622973334
---
Test MAE: 0.0582393821085595
Test RMSE: 0.1170410744618299
Test R2: 0.9255948705822018
Model saved to ./models/xgb_best_and_decoded_features.joblib

Top 20 features:
                         Feature  Importance
       AdjacentMEGNet_layer32_24    0.470572
       AdjacentMEGNet_layer32_28    0.178220
        AdjacentMEGNet_layer16_5    0.049051
       AdjacentMEGNet_layer32_13    0.032829
       AdjacentMEGNet_layer32_12    0.026732
        AdjacentMEGNet_layer16_6    0.026098
        AdjacentMEGNet_layer32_8    0.015247
       AdjacentMEGNet_layer16_14    0.011529
        AdjacentMEGNet_layer16_7    0.011104
               ORB_v3_layer_1_38    0.007158
       AdjacentMEGNet_layer32_26    0.006071
     ROOST_mpgap_LayerOutput_#13    0.004765
     ROOST_mpgap_LayerOutput_#01    0.004604
     ROOST_mpgap_LayerOutput_#33    0.004476
              ORB_v3_layer_1_125    0.004066
         XRDPow

#### We observe a slight increase in R² which we attribute to the most robust decoded features being present for learning.

#### We can now use the decoded features as input to compute our multipurpose SISSO formulas

In [7]:
from mattervial.featurizers.sisso_featurization import get_sisso_features
features_mm['target'] = 1
sisso_features = get_sisso_features(features_mm, type="SISSO_FORMULAS_v1")
display(sisso_features.head())

--- Starting GET SISSO FEATURES ---
Using type 'SISSO_FORMULAS_v1', formulas file set to: /gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/mattervial-0.1.4-py3.9.egg/mattervial/featurizers/formulas/SISSO_FORMULAS_v1.txt
Using type 'SISSO_FORMULAS_v1', robust scaler file set to: /gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/mattervial-0.1.4-py3.9.egg/mattervial/featurizers/formulas/robust_scaler_mpgap_for_sisso.json
Master Formulas File: /gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/mattervial-0.1.4-py3.9.egg/mattervial/featurizers/formulas/SISSO_FORMULAS_v1.txt
Robust Scaler Params: /gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/mattervial-0.1.4-py3.9.egg/mattervial/featurizers/formulas/robust_scaler_mpgap_for_sisso.json
Loading data...
Using provided DataFrame
Feature columns identified: ['AtomicOrbitals|HOMO_character', 'AtomicOrbitals|HOMO_eleme

,SISSO_matbench_dielectric_3,SISSO_matbench_phonons_1,SISSO_matbench_phonons_2,SISSO_matbench_phonons_3,SISSO_matbench_phonons_4,SISSO_matbench_phonons_5,SISSO_matbench_phonons_6,SISSO_matbench_phonons_7,SISSO_matbench_phonons_8,SISSO_matbench_phonons_9,...,SISSO_matbench_expt_gap_11,SISSO_matbench_expt_gap_12,SISSO_matbench_expt_gap_13,SISSO_matbench_expt_gap_14,SISSO_matbench_expt_gap_15,SISSO_matbench_expt_gap_16,SISSO_matbench_expt_gap_17,SISSO_matbench_expt_gap_18,SISSO_matbench_expt_gap_19,SISSO_matbench_expt_gap_20
mp-624234,-0.001770,0.336329,0.326289,0.367281,0.339765,0.260636,0.385578,0.222679,0.312417,0.316484,...,-0.077620,-0.854417,1.587530e-04,0.998534,0.002932,0.025288,-0.002033,0.019135,0.015529,-0.001878
mp-560478,-1.246779,2.175993,2.190084,2.080899,2.040009,2.193271,2.665236,1.516152,1.996731,2.015382,...,1.111098,0.246824,3.973713e-05,0.999418,0.001164,0.007505,0.014299,0.005542,0.003479,0.014368
mp-556346,-0.819238,0.937851,0.953936,0.912556,0.881874,1.021082,1.020397,0.565125,0.899883,0.886216,...,0.007571,-0.460387,1.424437e-07,0.999986,0.000027,0.003545,0.000609,0.002706,0.002064,0.000413
mp-13676,-0.062186,2.896264,2.918785,2.785004,2.683853,2.802409,3.411747,1.871408,2.631124,2.637944,...,0.329330,-0.883216,1.311327e-08,0.999997,0.000006,0.000638,-0.000276,0.000449,0.000221,-0.000237
mp-7610,-0.791605,1.963798,1.885702,1.939934,1.719730,1.782847,1.903341,1.166132,1.409842,1.443448,...,-0.033628,-0.735307,7.378219e-06,0.999811,0.000379,0.011164,0.010691,0.008119,0.005994,0.010347


In [8]:
import pandas as pd
import numpy as np
import os, joblib
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('data/df_ref_index.csv',index_col=0)
# Load the feature dataframes
features_l_mm = pd.read_csv('./data/ref_index_features_l_mm.csv', index_col=0)
features_l_ofm = pd.read_csv('./data/ref_index_features_l_ofm.csv', index_col=0)
features_mvl = pd.read_csv('./data/ref_index_features_mvl.csv', index_col=0)
adjmegnet_features = pd.read_csv('./data/ref_index_features_adjmegnet.csv', index_col=0)
roost_gap_features = pd.read_csv('./data/ref_index_features_roost_gap.csv', index_col=0)
roost_eform_features = pd.read_csv('./data/ref_index_features_roost_eform.csv', index_col=0)
orb_features = pd.read_csv('./data/ref_index_features_orbv3.csv', index_col=0)
all_features = [features_l_mm, features_l_ofm, features_mvl, adjmegnet_features, roost_gap_features, roost_eform_features, orb_features, features_ofm, features_mm]
all_features.append(sisso_features)
# Concatenate all features (aligns by index)
X_all = pd.concat(all_features, axis=1)
# X_all = pd.concat([ features_mvl, adjmegnet_features, roost_gap_features, roost_eform_features, orb_features, features_ofm, features_mm], axis=1)

## We need to separate in train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Align to train/test indexes and fill missing values
X_train = X_all.reindex(train_df.index).fillna(0)
X_test = X_all.reindex(test_df.index).fillna(0)

y_train = train_df['ref_index']
y_test = test_df['ref_index']

# Train XGBoost regressor
model = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
model.fit(X_train.values, y_train.values)

# Predict and report metrics
y_pred_train = model.predict(X_train.values)
y_pred_test = model.predict(X_test.values)

print("Train MAE:", mean_absolute_error(y_train, y_pred_train))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train)))
print("Train R2:", r2_score(y_train, y_pred_train))
print("---")
print("Test MAE:", mean_absolute_error(y_test, y_pred_test))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
print("Test R2:", r2_score(y_test, y_pred_test))

# Save model
os.makedirs('./models', exist_ok=True)
model_path = './models/xgb_best_and_decoded_features.joblib'
joblib.dump(model, model_path)
print("Model saved to", model_path)

# Show top 20 feature importances
fi = pd.DataFrame({
   'Feature': X_train.columns,
   'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 20 features:")
print(fi.head(20).to_string(index=False))


Train MAE: 0.0011784430574798886
Train RMSE: 0.0015460128587858827
Train R2: 0.9999867991019861
---
Test MAE: 0.06051734473720846
Test RMSE: 0.11655276607243599
Test R2: 0.9262144284109011
Model saved to ./models/xgb_best_and_decoded_features.joblib

Top 20 features:
                    Feature  Importance
  AdjacentMEGNet_layer32_24    0.495793
  AdjacentMEGNet_layer32_28    0.093896
   AdjacentMEGNet_layer16_5    0.051646
   AdjacentMEGNet_layer16_6    0.040689
  AdjacentMEGNet_layer32_13    0.031595
  AdjacentMEGNet_layer32_12    0.030143
  AdjacentMEGNet_layer32_26    0.022397
   AdjacentMEGNet_layer16_7    0.017378
   AdjacentMEGNet_layer32_8    0.014189
  AdjacentMEGNet_layer16_14    0.012147
ROOST_mpgap_LayerOutput_#01    0.009700
          ORB_v3_layer_1_38    0.007956
   AdjacentMEGNet_layer16_9    0.006720
   AdjacentMEGNet_layer32_5    0.004855
         ORB_v3_layer_1_125    0.004812
ROOST_mpgap_LayerOutput_#33    0.004727
          ElementFraction|N    0.004219
    XRDPowde

### We can see again a slight improvement in our metrics as we introduce these more robust features.

#### In the following we exemplify how we can decode some of the Adjacent model features using symbolic regression. Our recommended tool however is to use SISSO instead of pySR.

In [9]:
import pandas as pd
import numpy as np
from pysr import PySRRegressor
import re

# Filter the two sets of features you want to use
X_train_MM = X_train.filter(like='|')
X_train_OFM = X_train.filter(like='OFM:')

# Combine them into a single DataFrame first
X = pd.concat([X_train_MM, X_train_OFM], axis=1)
y = X_train['AdjacentMEGNet_layer32_24']

# --- NEW: Clean all column names in the final DataFrame ---
# This single line replaces all your previous .str.replace() calls
# It replaces any character that is NOT a letter, number, or underscore with '_'
X.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X.columns]

# Check a few cleaned columns to confirm
print("Example of cleaned column names:")
print(X.columns[:5].tolist())
print("-" * 20)


# 2. Define the model
model_sympy = PySRRegressor(
    niterations=10,
    binary_operators=["+", "*", "/", "-", "pow"], # Added power operator
    unary_operators=[ "exp", "log", "square", "cube", "sqrt"], # More building blocks
    model_selection="best",
)

# 3. Fit the model to find the equation
# The cleaned X DataFrame is now used
model_sympy.fit(X, y)

# 4. Print the discovered equation
print("\nDiscovered Equation:")
print(model_sympy)

# You can also get the equation as a sympy object
print("\nSympy format:")
print(model_sympy.sympy())

/gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...


Example of cleaned column names:
['AtomicOrbitals_HOMO_character', 'AtomicOrbitals_HOMO_element', 'AtomicOrbitals_HOMO_energy', 'AtomicOrbitals_LUMO_character', 'AtomicOrbitals_LUMO_element']
--------------------


/gpfs/home/acad/ucl-modl/rgouvea/miniconda3/envs/ML39/lib/python3.9/site-packages/pysr/sr.py:96: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://ai.damtp.cam.ac.uk/pysr/tuning/
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 9.100e+03
Progress: 54 / 310 total iterations (17.419%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.430e-02  0.000e+00  y = 0.033603
2           1.360e-02  4.966e-02  y = square(OPSiteFingerprint_mean_cuboctahedral_CN_12)
3           1.254e-02  8.167e-02  y = XRDPowderPattern_xrd_30 - AGNIFingerPrint_std_dev_AGNI...
                                      _dir_z_eta_8_00e_01
5           1.019e-02  1.038e-01  y = sqrt(square(-0.16463)) - AGNIFingerPrint_std_dev_AGNI_...
                                      dir_x_eta_1_88e_00
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <e

[ Info: Final population:
[ Info: Results saved to:



Discovered Equation:
PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          0.0336037   
	1        0.250363    square(ValenceOrbital_frac_d_valence_electrons)   
	2        0.091141  0.16967736 - AGNIFingerPrint_std_dev_AGNI_dir_...   
	3        0.007198  sqrt(XRDPowderPattern_xrd_30) - AGNIFingerPrin...   
	4        0.010627  (0.1696653 - AGNIFingerPrint_std_dev_AGNI_dir_...   
	5  >>>>  0.129105  OPSiteFingerprint_mean_pentagonal_planar_CN_5 ...   
	6        0.004250  (OPSiteFingerprint_mean_pentagonal_planar_CN_5...   
	7        0.011958  OPSiteFingerprint_mean_pentagonal_planar_CN_5 ...   
	
	       loss  complexity  
	0  0.014297           1  
	1  0.011131           2  
	2  0.010161           3  
	3  0.010088           4  
	4  0.009982           5  
	5  0.007710           7  
	6  0.007645           9  
	7  0.007288          13  
]

Sympy format:
OPSiteFingerprint_mean_pentago

In [10]:
# Get all discovered equations in a DataFrame
equations_df = model_sympy.equations_

# Sort them by loss (lower is better)
equations_sorted = equations_df.sort_values("loss", ascending=True)

# Select top 10 formulas by accuracy (ignoring complexity)
top10 = equations_sorted.head(10)

In [17]:
import sympy
import pandas as pd
import numpy as np

def create_topk_pysr_features(pysr_model, data_df, k=10):
    """
    Evaluates the top-k best-performing PySR equations (ignoring complexity)
    on a given DataFrame. Returns a DataFrame with one column per equation.
    """
    # Get all discovered equations
    equations_df = pysr_model.equations_

    # Sort them by loss (lower is better) and take top-k
    top_equations = equations_df.sort_values("loss", ascending=True).head(k)

    # Create an empty DataFrame to store the new features
    new_features_df = pd.DataFrame(index=data_df.index)

    print(f"Generating {len(top_equations)} new features...")

    # Loop through each discovered equation
    for rank, (i, row) in enumerate(top_equations.iterrows(), start=1):
        # Get the equation as a sympy object
        expr = pysr_model.sympy(i)
        
        # Get the symbols (variable names) required by this specific equation
        symbols = sorted([str(s) for s in expr.free_symbols])
        
        # If there are no symbols, it's a constant. Handle this case.
        if not symbols:
            feature_name = f"pysr_top{rank}_const"
            new_features_df[feature_name] = np.full(len(data_df), float(expr))
            continue

        # Create a fast, callable function from the sympy expression
        callable_func = sympy.lambdify(symbols, expr, 'numpy')
        
        # Prepare the input data for the function in the correct order
        input_data = [data_df[sym].values for sym in symbols]

        # Evaluate the function on the data and store it in the new DataFrame
        feature_name = f"pysr_top{rank}"
        new_features_df[feature_name] = callable_func(*input_data)

    return new_features_df

# Example usage:
X_all.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X_all.columns]

X_pysr_features = create_topk_pysr_features(model_sympy, X_all, k=10)
display(X_pysr_features)


Generating 8 new features...


,pysr_top1,pysr_top2,pysr_top3,pysr_top4,pysr_top5,pysr_top6,pysr_top7,pysr_top8_const
mp-624234,0.078162,0.087834,0.088045,0.049400,0.100539,0.061420,0.065203,0.033604
mp-560478,-0.016115,-0.007906,-0.009011,0.009971,0.044924,0.012407,0.003764,0.033604
mp-556346,0.001893,0.011582,0.011461,0.008893,0.018363,0.011067,0.050265,0.033604
mp-13676,-0.025266,-0.007521,-0.005279,-0.029453,-0.031449,-0.036601,0.028966,0.033604
mp-7610,0.017846,0.024978,0.023266,0.036307,0.000557,0.045144,0.004683,0.033604
...,...,...,...,...,...,...,...,...
mp-8960,0.057970,0.064678,0.061685,0.066864,0.117386,0.083128,0.009857,0.033604
mp-558257,0.008822,0.016849,0.018006,0.017256,0.037638,0.021462,0.035506,0.033604
mp-770408,0.011039,0.019760,0.015545,0.032356,0.088407,0.040233,0.005030,0.033604
mp-722684,-0.104224,-0.086242,-0.085904,-0.078977,-0.101313,-0.098162,0.000493,0.033604
